In [4]:
from deepchecks.core import CheckResult, ConditionCategory, ConditionResult
from deepchecks.tabular import TrainTestCheck, Dataset, Context
from sklearn.metrics import precision_score
import numpy as np

class PrecisionCheck(TrainTestCheck):
    """Custom Deepchecks check for evaluating model precision."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def run_logic(self, context: Context) -> CheckResult:
        """Computes precision score using model predictions on test data."""
        model = context.model
        test_dataset = context.test  # Get test dataset

        if model is None or test_dataset is None:
            return CheckResult(False, display="Model and test dataset are required.")

        if test_dataset.label_name is None:
            return CheckResult(False, display="Test dataset must have labels.")

        # Extract true labels and predictions
        y_true = test_dataset.data[test_dataset.label_name].values
        y_pred = model.predict(test_dataset.data[test_dataset.features])

        # Compute precision
        precision = precision_score(y_true, y_pred, average='binary')

        # Return CheckResult with the computed precision score
        return CheckResult(value=precision, display=f'Precision: {precision:.4f}')

    def add_condition_precision_above(self, threshold: float = 0.80):
        """Adds a condition to ensure precision is above 0.80."""

        def condition(result_value):
            """Condition function to check if precision is above the threshold."""
            if result_value >= threshold:
                return ConditionResult(ConditionCategory.PASS, f"Precision {result_value:.4f} meets/exceeds {threshold}")
            else:
                return ConditionResult(ConditionCategory.FAIL, f"Precision {result_value:.4f} is below {threshold}")

        # Add the condition to the check instance
        return self.add_condition(f'Precision ≥ {threshold}', condition)


# Import dependencies for testing the check
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Generate a sample dataset
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Convert train and test sets to Deepchecks Dataset
train_dataset = Dataset(X_train, label=y_train)
test_dataset = Dataset(X_test, label=y_test)

# Create the precision check instance
precision_check = PrecisionCheck().add_condition_precision_above(0.99)

# Run the check using train and test datasets
result = precision_check.run(train_dataset, test_dataset, model=model)

# Show the result
result.show()


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.


In [5]:
from deepchecks.core import CheckResult, ConditionCategory, ConditionResult
from deepchecks.tabular import TrainTestCheck, Dataset
from sklearn.metrics import precision_score
import numpy as np

class PrecisionCheck(TrainTestCheck):
    """Custom Deepchecks check for evaluating model precision on both train and test datasets."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def run_logic(self, context) -> CheckResult:
        """Computes precision score for both train and test datasets."""
        model = context.model
        train_dataset = context.train  # Get train dataset
        test_dataset = context.test  # Get test dataset

        if model is None or train_dataset is None or test_dataset is None:
            return CheckResult(False, display="Model, train dataset, and test dataset are required.")

        if train_dataset.label_name is None or test_dataset.label_name is None:
            return CheckResult(False, display="Both train and test datasets must have labels.")

        # Compute precision for train dataset
        y_train_true = train_dataset.data[train_dataset.label_name].values
        y_train_pred = model.predict(train_dataset.data[train_dataset.features])
        train_precision = precision_score(y_train_true, y_train_pred, average='binary')

        # Compute precision for test dataset
        y_test_true = test_dataset.data[test_dataset.label_name].values
        y_test_pred = model.predict(test_dataset.data[test_dataset.features])
        test_precision = precision_score(y_test_true, y_test_pred, average='binary')

        # Display results
        display_str = (f'Train Precision: {train_precision:.4f} \n'
                       f'Test Precision: {test_precision:.4f}')

        return CheckResult(value={"train_precision": train_precision, "test_precision": test_precision}, display=display_str)

    def add_condition_precision_above(self, threshold: float = 0.80):
        """Adds a condition to ensure precision is above a given threshold for both train and test datasets."""

        def condition(result_value):
            """Condition function to check if precision is above the threshold."""
            train_precision = result_value["train_precision"]
            test_precision = result_value["test_precision"]

            train_status = "PASS" if train_precision >= threshold else "FAIL"
            test_status = "PASS" if test_precision >= threshold else "FAIL"

            message = (f"Train Precision: {train_precision:.4f} ({train_status})\n"
                       f"Test Precision: {test_precision:.4f} ({test_status})")

            if train_precision >= threshold and test_precision >= threshold:
                return ConditionResult(ConditionCategory.PASS, message)
            else:
                return ConditionResult(ConditionCategory.FAIL, message)

        return self.add_condition(f'Precision ≥ {threshold} for both train & test', condition)


# Import dependencies for testing the check
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Generate a sample dataset
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Convert train and test sets to Deepchecks Dataset
train_dataset = Dataset(X_train, label=y_train)
test_dataset = Dataset(X_test, label=y_test)

# Create the precision check instance
precision_check = PrecisionCheck().add_condition_precision_above(0.80)

# Run the check using train and test datasets
result = precision_check.run(train_dataset, test_dataset, model=model)

# Show the result
result.show()


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.


In [6]:
from deepchecks.core import CheckResult, ConditionCategory, ConditionResult
from deepchecks.tabular import TrainTestCheck, Dataset
from sklearn.metrics import precision_score

class PrecisionCheck(TrainTestCheck):
    """Custom Deepchecks check for evaluating precision with or without a model."""

    def __init__(self, use_model=True, y_train_pred=None, y_test_pred=None, **kwargs):
        """
        use_model: If True, use the model for prediction. If False, rely on provided predicted labels.
        y_train_pred: Predicted labels for train set (if use_model=False).
        y_test_pred: Predicted labels for test set (if use_model=False).
        """
        super().__init__(**kwargs)
        self.use_model = use_model  # Control whether to use the model
        self.y_train_pred = y_train_pred
        self.y_test_pred = y_test_pred

    def run_logic(self, context) -> CheckResult:
        """Computes precision score for both train and test datasets."""
        train_dataset = context.train
        test_dataset = context.test
        model = context.model if self.use_model else None  # Use model only if enabled

        if train_dataset is None or test_dataset is None:
            return CheckResult(False, display="Train and test datasets are required.")

        if train_dataset.label_name is None or test_dataset.label_name is None:
            return CheckResult(False, display="Both train and test datasets must have actual labels.")

        # Extract true labels
        y_train_true = train_dataset.data[train_dataset.label_name].values
        y_test_true = test_dataset.data[test_dataset.label_name].values

        # Get predictions: Either from the model or from provided predicted labels
        if self.use_model:
            if model is None:
                return CheckResult(False, display="Model is required but not provided.")
            y_train_pred = model.predict(train_dataset.data[train_dataset.features])
            y_test_pred = model.predict(test_dataset.data[test_dataset.features])
        else:
            if self.y_train_pred is None or self.y_test_pred is None:
                return CheckResult(False, display="Predicted labels for train and test must be provided when use_model=False.")
            y_train_pred = self.y_train_pred
            y_test_pred = self.y_test_pred

        # Compute precision
        train_precision = precision_score(y_train_true, y_train_pred, average='binary')
        test_precision = precision_score(y_test_true, y_test_pred, average='binary')

        display_str = (f'Train Precision: {train_precision:.4f} \n'
                       f'Test Precision: {test_precision:.4f}')

        return CheckResult(value={"train_precision": train_precision, "test_precision": test_precision}, display=display_str)

    def add_condition_precision_above(self, threshold: float = 0.80):
        """Adds a condition to ensure precision is above a given threshold."""

        def condition(result_value):
            train_precision = result_value["train_precision"]
            test_precision = result_value["test_precision"]

            train_status = "PASS" if train_precision >= threshold else "FAIL"
            test_status = "PASS" if test_precision >= threshold else "FAIL"

            message = (f"Train Precision: {train_precision:.4f} ({train_status})\n"
                       f"Test Precision: {test_precision:.4f} ({test_status})")

            if train_precision >= threshold and test_precision >= threshold:
                return ConditionResult(ConditionCategory.PASS, message)
            else:
                return ConditionResult(ConditionCategory.FAIL, message)

        return self.add_condition(f'Precision ≥ {threshold} for both train & test', condition)


# Import dependencies for testing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np

# Generate a sample dataset
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Simulate predicted labels (e.g., from a pre-saved model output)
y_train_pred = np.random.choice([0, 1], size=len(y_train))  # Simulated train predictions
y_test_pred = np.random.choice([0, 1], size=len(y_test))  # Simulated test predictions

# Convert train and test sets to Deepchecks Dataset
train_dataset = Dataset(X_train, label=y_train)
test_dataset = Dataset(X_test, label=y_test)

# Option 1: Run the check using a model
precision_check_model = PrecisionCheck(use_model=True).add_condition_precision_above(0.80)
result_model = precision_check_model.run(train_dataset, test_dataset, model=model)
result_model.show()

# Option 2: Run the check using only predicted labels (no model)
precision_check_no_model = PrecisionCheck(use_model=False, y_train_pred=y_train_pred, y_test_pred=y_test_pred)
precision_check_no_model.add_condition_precision_above(0.80)
result_no_model = precision_check_no_model.run(train_dataset, test_dataset)
result_no_model.show()


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.


In [7]:
# Option 2: Run the check using only predicted labels (no model)
precision_check_no_model = PrecisionCheck(use_model=False, y_train_pred=y_train_pred, y_test_pred=y_test_pred)
precision_check_no_model.add_condition_precision_above(0.80)
result_no_model = precision_check_no_model.run(train_dataset, test_dataset)
result_no_model.show()

In [10]:
model_eval_suite = Suite("Model Evaluation Suite")
model_eval_suite.add(precision_check)

# Run the suite
suite_result = model_eval_suite.run(train_dataset, test_dataset, model=model)

# Show results
suite_result.show()

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_F3JG4RG9XBWR6AZSDVNK122L3">Model Evaluation S…